<h1 align="center"><u>Spring 2017 - CMPT 733 - Programming in Big Data - II</u></h3>
<h2 align="center">Final Project - Analyzing Immigration in the U.S.A.</h3>
<h3 align="center">Analysis of <a href="https://www.kaggle.com/nsharan/h-1b-visa"> H-1B dataset</a> from Kaggle</h3>
<h3 align="center">Author : Kavan Shukla</h3>

## Data Cleaning:

In [1]:
import numpy as np
import pandas as pd
from IPython.display import Markdown, display

def printme(string):
    display(Markdown(string))
data = pd.read_csv('h1b_kaggle.csv')
data = data.dropna()
data.reset_index()

data.loc[:,'WORKSITE']=data.loc[:,'WORKSITE'].apply(lambda rec:rec.split(',')[1][1:])

def chgNa(rec):           
    if (rec=='NA'):         
        return 'Someplace'
    return rec

data.loc[:,'WORKSITE']=data.loc[:,'WORKSITE'].apply(lambda rec: chgNa(rec))

data.columns = ["RECORD", "CASE_STATUS", "EMPLOYER_NAME", 
                            "SOC_NAME", "JOB_TITLE", "FULL_TIME_POSITION", "PREVAILING_WAGE", "YEAR", "WORKSITE", "lon", "lat"]


In [2]:
data.drop('lon', axis=1, inplace=True)
data.drop('lat', axis=1, inplace=True)
data['YEAR'] = data['YEAR'].astype(int)
data['PREVAILING_WAGE'] = data['PREVAILING_WAGE'].astype(long)
print len(data)
data.info()

2877765
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2877765 entries, 0 to 3002444
Data columns (total 9 columns):
RECORD                int64
CASE_STATUS           object
EMPLOYER_NAME         object
SOC_NAME              object
JOB_TITLE             object
FULL_TIME_POSITION    object
PREVAILING_WAGE       int64
YEAR                  int32
WORKSITE              object
dtypes: int32(1), int64(2), object(6)
memory usage: 208.6+ MB


In [3]:
data.head()

,RECORD,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067,2016,MICHIGAN
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674,2016,TEXAS
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066,2016,NEW JERSEY
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314,2016,COLORADO
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518,2016,MISSOURI


In [4]:
printme("**Employers with most petitions filed along with Petition Count:**")
employerPetitions = data['EMPLOYER_NAME'].value_counts().nlargest(20)
print employerPetitions
employerPetitions.to_csv('top20_employers_by_#ofPetitions_filed.csv', sep=',', encoding='utf-8')

**Employers that have filed maximum number of petitions with Petition Count:**

INFOSYS LIMITED                                    130257
TATA CONSULTANCY SERVICES LIMITED                   64273
WIPRO LIMITED                                       45673
DELOITTE CONSULTING LLP                             35999
IBM INDIA PRIVATE LIMITED                           33585
ACCENTURE LLP                                       33226
MICROSOFT CORPORATION                               25551
HCL AMERICA, INC.                                   22488
ERNST & YOUNG U.S. LLP                              18094
COGNIZANT TECHNOLOGY SOLUTIONS U.S. CORPORATION     17322
CAPGEMINI AMERICA INC                               16628
GOOGLE INC.                                         16425
LARSEN & TOUBRO INFOTECH LIMITED                    14318
IGATE TECHNOLOGIES INC.                             12327
IBM CORPORATION                                     11976
INTEL CORPORATION                                   11390
TECH MAHINDRA (AMERICAS),INC.                       10595
DELOITTE & TOU

In [5]:
printme ("**States filing the most number of H-1B petitions in decreasing order:**")
top_filing_states = data.groupby('WORKSITE').size().sort_values(ascending=0).head(10)
print top_filing_states

top_filing_states.to_csv('top10_states_by_#ofPetitions_filed.csv', sep=',', encoding='utf-8')

printme ("**States with most number of H-1B petitions approved in decreasing order**")
top_filing_states_approved = data[data['CASE_STATUS']=='CERTIFIED'].groupby('WORKSITE').size().sort_values(ascending=0).head(10)
print top_filing_states_approved

top_filing_states_approved.to_csv('top10_states_by_#ofPetitions_approved.csv', sep=',', encoding='utf-8')

**States filing the most number of H-1B petitions in decreasing order:**

WORKSITE
CALIFORNIA       548539
TEXAS            286704
NEW YORK         282429
NEW JERSEY       206876
ILLINOIS         154843
MASSACHUSETTS    112659
PENNSYLVANIA     103729
WASHINGTON       100754
FLORIDA          100384
GEORGIA           99679
dtype: int64


**States with most number of H-1B petitions approved in decreasing order**

WORKSITE
CALIFORNIA       473419
TEXAS            248518
NEW YORK         242474
NEW JERSEY       184982
ILLINOIS         137347
MASSACHUSETTS     97051
PENNSYLVANIA      92825
GEORGIA           88620
WASHINGTON        88421
FLORIDA           87318
dtype: int64


In [6]:
printme ("**Top 20 Job titles by number of petitions filed:**")
jobTitles = data['JOB_TITLE'].value_counts().nlargest(20)
print jobTitles
jobTitles.to_csv('top20_JobTitles_by_#ofPetitions_filed.csv', sep=',', encoding='utf-8')

**Top 20 Job titles by number of petitions filed:**

PROGRAMMER ANALYST             243357
SOFTWARE ENGINEER              118897
COMPUTER PROGRAMMER             68696
SYSTEMS ANALYST                 60754
SOFTWARE DEVELOPER              41875
BUSINESS ANALYST                38781
COMPUTER SYSTEMS ANALYST        34036
TECHNOLOGY LEAD - US            28307
SENIOR SOFTWARE ENGINEER        26617
TECHNOLOGY ANALYST - US         26010
ASSISTANT PROFESSOR             24436
SENIOR CONSULTANT               24120
CONSULTANT                      22643
PROJECT MANAGER                 19015
DATABASE ADMINISTRATOR          16108
PHYSICAL THERAPIST              14203
RESEARCH ASSOCIATE              13409
COMPUTER PROGRAMMER ANALYST     13116
ACCOUNTANT                      12934
DEVELOPER                       12737
Name: JOB_TITLE, dtype: int64


In [7]:
printme ('**Year wise petitions status count:**')
caseStatusPerYear = data.groupby(['YEAR','CASE_STATUS']).count()
print caseStatusPerYear['RECORD']
caseStatusPerYear['RECORD'].to_csv('Petitions_yearly_CaseStatus.csv', sep=',', encoding='utf-8')

**Year wise petitions status count:**

YEAR  CASE_STATUS                                       
2011  CERTIFIED                                             287638
      CERTIFIED-WITHDRAWN                                    10837
      DENIED                                                 25986
      WITHDRAWN                                               9164
2012  CERTIFIED                                             335676
      CERTIFIED-WITHDRAWN                                    29728
      DENIED                                                 18866
      WITHDRAWN                                               9997
2013  CERTIFIED                                             366384
      CERTIFIED-WITHDRAWN                                    34126
      DENIED                                                 10976
      PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED        15
      WITHDRAWN                                              10926
2014  CERTIFIED                                             436985
     

In [8]:
printme ("**Job Titles with Maximum wages over last 6 years:**")
max_wages_job_titles = data.groupby(['JOB_TITLE'])['PREVAILING_WAGE'].min().sort_values(ascending=0).head(10)
print max_wages_job_titles

**Job Titles with Maximum wages over last 6 years:**

JOB_TITLE
DIRECTOR, SOCIAL AND DIGITAL MEDIA                    329139200
DIRECTOR OF SRCH ENGINE OPTMZTN/SRCH ENGINE MKTNG     245937120
ASSOCIATE DIRECTOR, HEALTH ECONOMICS AND OUTCOMES     221424320
AREA MANAGER, PHARMACEUTICAL PACKAGING                212987840
SR. WEB INFRASTRUCTURE CONSULT                        208012480
PROGRAMMER ANALYST/SAP SRM ANALYST                    207105600
DEVELOPER (SOFTWARE SYSTEMS APPLICATIONS)             176560800
STAFF CONSULTANT - MICRO                              169507520
QAULITY TEST ENGINEER                                 155879360
SLAMBERTPA@AOL.COM                                    144589120
Name: PREVAILING_WAGE, dtype: int64


In [9]:
printme ("**Yearly average wages for H-1B class employee:**")
h1b_wages_yearly_trend = data.groupby(['YEAR']).mean()
print h1b_wages_yearly_trend['PREVAILING_WAGE']
h1b_wages_yearly_trend['PREVAILING_WAGE'].to_csv('h1b_wages_yearly_trend.csv', sep=',', encoding='utf-8')

**Yearly average wages for H-1B class employee:**

YEAR
2011    194288
2012    176106
2013    194009
2014    181762
2015     91661
2016     89016
Name: PREVAILING_WAGE, dtype: int64


In [10]:
printme ("**Yearly Highest paying employer for DATA SCIENTIST Job Position:**")
data_scientist_yearly_max_wage = data[(data['JOB_TITLE']=='DATA SCIENTIST') &  (data['CASE_STATUS']=='CERTIFIED')].groupby(['YEAR']).max()
print data_scientist_yearly_max_wage[['EMPLOYER_NAME', 'PREVAILING_WAGE']]
data_scientist_yearly_max_wage[['EMPLOYER_NAME', 'PREVAILING_WAGE']].to_csv('data_scientist_yearly_max_wage.csv', sep=',', encoding='utf-8')

**Yearly Highest paying employer for DATA SCIENTIST Job Position:**

          EMPLOYER_NAME  PREVAILING_WAGE
YEAR                                    
2011       TAGGED, INC.           105394
2012  ZIONS BANK / ZMSC           139256
2013        YAHOO! INC.           157435
2014         ZYNGA INC.           152776
2015     ZYPMEDIA, INC.           157602
2016         ZYNGA INC.           165006


In [11]:
printme ("**Yearly Average wages for DATA SCIENTIST Job Position:**")
data_scientist_yearly_mean_wage = data[(data['JOB_TITLE']=='DATA SCIENTIST') &  (data['CASE_STATUS']=='CERTIFIED')].groupby(['YEAR']).mean()
print data_scientist_yearly_mean_wage['PREVAILING_WAGE']
data_scientist_yearly_mean_wage['PREVAILING_WAGE'].to_csv('data_scientist_yearly_mean_wage.csv', sep=',', encoding='utf-8')

**Yearly Average wages for DATA SCIENTIST Job Position:**

YEAR
2011    85508.875000
2012    91321.315789
2013    90072.761468
2014    90284.222727
2015    91001.748031
2016    86628.140394
Name: PREVAILING_WAGE, dtype: float64


In [12]:
printme("**States filing the most number of H-1B petitions in decreasing order:**")
Job_titles_count = data.groupby('JOB_TITLE').size().sort_values(ascending=0).head(20)
print Job_titles_count
Job_titles_count.to_csv('Job_titles_count.csv', sep=',', encoding='utf-8')

**States filing the most number of H-1B petitions in decreasing order:**

JOB_TITLE
PROGRAMMER ANALYST             243357
SOFTWARE ENGINEER              118897
COMPUTER PROGRAMMER             68696
SYSTEMS ANALYST                 60754
SOFTWARE DEVELOPER              41875
BUSINESS ANALYST                38781
COMPUTER SYSTEMS ANALYST        34036
TECHNOLOGY LEAD - US            28307
SENIOR SOFTWARE ENGINEER        26617
TECHNOLOGY ANALYST - US         26010
ASSISTANT PROFESSOR             24436
SENIOR CONSULTANT               24120
CONSULTANT                      22643
PROJECT MANAGER                 19015
DATABASE ADMINISTRATOR          16108
PHYSICAL THERAPIST              14203
RESEARCH ASSOCIATE              13409
COMPUTER PROGRAMMER ANALYST     13116
ACCOUNTANT                      12934
DEVELOPER                       12737
dtype: int64


In [13]:
data_ML = data.copy()
for col in ['CASE_STATUS', 'EMPLOYER_NAME', 'SOC_NAME', 'JOB_TITLE', 'FULL_TIME_POSITION','WORKSITE']:
    data_ML[col] = data_ML[col].astype('category')
data_ML.head()

,RECORD,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067,2016,MICHIGAN
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674,2016,TEXAS
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066,2016,NEW JERSEY
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314,2016,COLORADO
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518,2016,MISSOURI


In [14]:
printme('**Categorized Dataframe:**')
cat_columns = data_ML.select_dtypes(['category']).columns
data_ML[cat_columns] = data_ML[cat_columns].apply(lambda x: x.cat.codes)
data_ML.head()

**Categorized Dataframe:**

,RECORD,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE
0,1,1,200035,165,155638,0,36067,2016,22
1,2,1,78574,248,41362,1,242674,2016,45
2,3,1,148855,248,41431,1,193066,2016,30
3,4,1,73925,248,178343,1,220314,2016,5
4,5,6,144243,248,156922,1,157518,2016,25


In [15]:
printme('**Saved Categorized dataframe to .csv file**')
data_ML.to_csv('data_ML.csv',encoding='utf-8')

**Saving Categorized dataframe to .csv file**

# Logistic Regression Model (Multiclass):
### Predicting H-1B eligibility(Case_Status Outcome) given a profile with following attributes: 
#### 'EMPLOYER_NAME', 'SOC_NAME', 'JOB_TITLE', 'FULL_TIME_POSITION', 'PREVAILING_WAGE' and 'WORKSITE'

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

train_split, test_split = train_test_split(data_ML, train_size=0.80, random_state=1)

data_ML_train = pd.DataFrame(train_split, columns=data_ML.columns)
data_ML_test = pd.DataFrame(test_split, columns=data_ML.columns)

X_train = data_ML_train[['EMPLOYER_NAME', 'SOC_NAME', 'JOB_TITLE', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'WORKSITE']]
y_train = data_ML_train['CASE_STATUS']
X_test = data_ML_test[['EMPLOYER_NAME', 'SOC_NAME', 'JOB_TITLE', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'WORKSITE']]
y_test = data_ML_test['CASE_STATUS']
lr_model = LogisticRegression(multi_class='multinomial', solver ='newton-cg')
lr_model.fit(X_train, y_train)
prediction = lr_model.predict(X_test)
print(metrics.classification_report(y_test, prediction))
print metrics.accuracy_score(y_test, prediction)

             precision    recall  f1-score   support

          0       0.87      1.00      0.93    502388
          1       0.00      0.00      0.00     39096
          2       0.87      0.01      0.03     17181
          4       0.00      0.00      0.00         3
          6       0.00      0.00      0.00     16885

avg / total       0.79      0.87      0.81    575553

0.873302719298


### Saving the trained Model

In [17]:
import pickle 
filename = "LRmodel_100.sav"
pickle.dump(lr_model, open(filename, 'wb'))

### Accuracy Score:

In [18]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.873302719298


In [19]:
printme ("**Logistic Regression Model (max_iter = 200) Results: **")
lr_model_200 = LogisticRegression(multi_class='multinomial', solver ='newton-cg', max_iter=200)
lr_model_200.fit(X_train, y_train)
prediction_200 = lr_model_200.predict(X_test)
print metrics.classification_report(y_test, prediction_200)
print metrics.accuracy_score(y_test, prediction_200)

**Logistic Regression Model (max_iter = 200) Results: **

             precision    recall  f1-score   support

          0       0.87      1.00      0.93    502388
          1       0.00      0.00      0.00     39096
          2       0.87      0.01      0.03     17181
          4       0.00      0.00      0.00         3
          6       0.00      0.00      0.00     16885

avg / total       0.79      0.87      0.81    575553

0.873304456757
